In [ ]:
# Libraries

In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import cv2 as cv

In [ ]:
from sklearn import linear_model

In [ ]:
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# Helping Functions

In [ ]:
def plot_image(image):
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.show(block=True)

In [ ]:
def display_image(image):
    cv.imshow('image', image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
def plot_graph(image, i=0):
    sum_ = image.sum(axis=i)
    plt.figure(figsize=(15, 8))
    x = list(range(len(sum_)))
    y = list(sum_)
    plt.scatter(x, y, alpha=0.5)
    plt.show()

In [ ]:
# Loading

In [ ]:
r_image = cv.imread('page03.jpg')

In [ ]:
gray = cv.cvtColor(r_image, cv.COLOR_BGR2GRAY)

In [ ]:
plot_image(r_image)

In [ ]:
# Preprocessing

In [ ]:
def preprocess_image(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    blur = cv.bilateralFilter(gray, 3, 75, 75)
    thresh = cv.adaptiveThreshold(blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    k = 3
    kernal = np.ones((k, k), np.uint8)
    dilate = cv.dilate(thresh, kernal, iterations=1)
    k = 1
    kernal = np.ones((k, k), np.uint8)
    erode = cv.erode(dilate, kernal, iterations=1)
    return erode

In [ ]:
preprocess_imaged = preprocess_image(r_image)

In [ ]:
plot_image(preprocess_imaged)